# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

## outline (for this course)

- OpenAi function calling
- LangChain Expression Language (LCEL)
- OpenAI function calling in LangChain
- Tagging and extraction using OpenAI function calling
- Tools and Routing
- Conversational Agent

## Function calling

- OpenAI has fine-tuned the gpt-3.5 turbo-0613 and gpt-4-0613 models to :
  1. accept additional arguments through which users can pass in descriptions of functions.
  2. if it is relevant, return the nam of the function to use, along with a JSON object with teh appropriate input parameters.


#### did not really follow the tutorials, as the syntex are basically completely outdated/different

#### base codes: https://platform.openai.com/docs/guides/function-calling

In [2]:
# !pip install python-dotenv
# !pip install openai
# !pip install -U langchain langchain-openai

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = openai.api_key

In [4]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
# old syntex. function will have to be incorporated into tools

# functions = [
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather in a given location",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
#             },
#             "required": ["location"],
#         },
#     }
# ]

In [5]:
# new syntex requires insert function into part of the tools

tools = [
    {
        "type": "function",
        "function":  {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
        }
    }
]

In [6]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [7]:
import openai

# The most basic version: text to text

In [8]:
# an example of new syntex from: https://github.com/openai/openai-python
import os
from openai import OpenAI

client = OpenAI(
    # This cannot be omitted
    api_key= OPENAI_API_KEY
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",

        }
    ],
    model="gpt-3.5-turbo",
)

In [9]:
chat_completion

ChatCompletion(id='chatcmpl-A9lrZhqG4TbBnF7NC6DNRqpl5czpy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726892301, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [39]:
chat_completion.choices[0].message.content

'1 + 1 equals 2.'

In [10]:
import os
from openai import OpenAI

client = OpenAI(
    # This cannot be omitted
    api_key= OPENAI_API_KEY
)

# the most basic version, to just ask a question in "content" and get the answer back in content.
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "tell me what 1 + 1 is?",
        }
    ],
    model="gpt-3.5-turbo",
)

In [11]:
chat_completion

ChatCompletion(id='chatcmpl-A9lsCpx8QCLg31MoB2FSWHQTOdzAx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1 + 1 is equal to 2.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726892340, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=17, total_tokens=27, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [38]:
chat_completion.choices[0].message.content

'1 + 1 equals 2.'

# The most basic version: text as prompt and image as the input

In [16]:
prompt = "tell me what is inside the picture?"

In [21]:
img_url = "https://picsum.photos/id/237/200/300"

In [22]:
# with a hosted image

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"{img_url}"},
                    # "image": {"path": "my_pic.jpg"},
                },
            ],
        }
    ],
)

In [23]:
response

ChatCompletion(id='chatcmpl-A9lzMUefSoX1SJcLk3aRczHhVNOVB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The picture shows a black puppy, likely a Labrador, resting on a wooden surface. The puppy has a shiny coat and expressive eyes.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726892784, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e9627b5346', usage=CompletionUsage(completion_tokens=27, prompt_tokens=8515, total_tokens=8542, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [37]:
response.choices[0].message.content

'The picture shows a beach scene with four individuals sitting on beach chairs, facing the ocean. They are positioned on a sandy shore with the water in front of them. The sea appears calm, and the sky is clear with some light clouds. Their chairs are colorful, and there are a few bags or items near them on the sand. The overall atmosphere suggests a relaxed day at the beach.'

## With the image as a base64 encoded string:

#### the reason we are doing this is we should convert an image into some machine readble text, aka bytes

In [27]:
import base64
from io import BytesIO
from PIL import Image

image = Image.open("my_pic.jpg")

# image to bytes
image_bytes = BytesIO()
image.save(image_bytes, format="JPEG")
image_bytes = image_bytes.getvalue()

# encode the image bytes to base64
base64_image = base64.b64encode(image_bytes).decode("utf-8")

# set the image type
img_type = "image/jpeg"

# mow base64_image contains the base64 string of the JPEG image


In [28]:
# img_type = "image/jpeg"
# base64_image = base64.b64encode(image_bytes.getvalue()).decode(
#         "utf-8")

In [29]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:{img_type};base64,{base64_image}"},
                },
            ],
        }
    ],
)

In [36]:
response

ChatCompletion(id='chatcmpl-A9m5XPVxR9dpMB9yeaS4G76l2PPPn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The picture shows a beach scene with four individuals sitting on beach chairs, facing the ocean. They are positioned on a sandy shore with the water in front of them. The sea appears calm, and the sky is clear with some light clouds. Their chairs are colorful, and there are a few bags or items near them on the sand. The overall atmosphere suggests a relaxed day at the beach.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893167, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e9627b5346', usage=CompletionUsage(completion_tokens=78, prompt_tokens=36850, total_tokens=36928, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [35]:
response.choices[0].message.content

'The picture shows a beach scene with four individuals sitting on beach chairs, facing the ocean. They are positioned on a sandy shore with the water in front of them. The sea appears calm, and the sky is clear with some light clouds. Their chairs are colorful, and there are a few bags or items near them on the sand. The overall atmosphere suggests a relaxed day at the beach.'

## Polling Helpers
When interacting with the API some actions such as starting a Run and adding files to vector stores are asynchronous and take time to complete. The SDK includes helper functions which will poll the status until it reaches a terminal state and then return the resulting object. If an API method results in an action that could benefit from polling there will be a corresponding version of the method ending in '_and_poll'.

For instance to create a Run and poll until it reaches a terminal state you can run:



# Async usage
Simply import AsyncOpenAI instead of OpenAI and use await with each API call:

In [48]:
import os
import asyncio
from openai import AsyncOpenAI

client = AsyncOpenAI(
    # cannot be omitted
    api_key= OPENAI_API_KEY
)

async def main() -> None:
    chat_completion = await client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Say this is a test",
            }
        ],
        model="gpt-3.5-turbo",
    )
    print(chat_completion)

if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        print("loop")
        # check if there's already a running event loop, use it
        task = loop.create_task(main())
    else:
        # otherwise, use asyncio.run()
        print("asyncio")
        asyncio.run(main())


loop


## Other stuff (still usable)
main source: https://platform.openai.com/docs/assistants/deep-dive/runs-and-run-steps

In [66]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, what is the weather like in Boston?"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # functions = functions,
    # function_call and functions go together
    # function_call="auto",
)

In [50]:
response

ChatCompletion(id='chatcmpl-A9mGH8qPyOYEkm87tB3qZF7HE904T', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I currently do not have access to real-time weather information. You can check a weather website or app like Weather.com or AccuWeather to get the latest weather updates for Boston.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893833, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=37, prompt_tokens=104, total_tokens=141, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [67]:
response.choices[0].message.content

"I'm currently unable to check the weather. You can use a weather website or app to get the latest updates."

In [64]:
# source: https://platform.openai.com/docs/guides/chat-completions/getting-started
# https://platform.openai.com/docs/api-reference/chat/streaming

from openai import OpenAI
client = OpenAI(
    api_key= OPENAI_API_KEY
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages= messages,
  # new syntex
  tools=tools,
  # function_call="auto"
)
response

ChatCompletion(id='chatcmpl-A9mHtijN9A6vswo1i9Po4tzg5LVcs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please provide me with your order ID so I can check the delivery date for you.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893933, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=18, prompt_tokens=107, total_tokens=125, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [65]:
response.choices[0].message.content

'Please provide me with your order ID so I can check the delivery date for you.'

In [62]:
# source: https://platform.openai.com/docs/guides/chat-completions/getting-started
# https://platform.openai.com/docs/api-reference/chat/streaming

from openai import OpenAI
client = OpenAI(
    api_key= OPENAI_API_KEY
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

response

ChatCompletion(id='chatcmpl-A9mHlOfIst3csW8RigJtMqronyzZG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas. This was the first time the World Series was held at a neutral site due to the COVID-19 pandemic.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893925, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=39, prompt_tokens=53, total_tokens=92, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [63]:
response.choices[0].message.content

'The 2020 World Series was played at Globe Life Field in Arlington, Texas. This was the first time the World Series was held at a neutral site due to the COVID-19 pandemic.'

In [56]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
# # // highlight-start
# does not work if you keep adding
# messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
# messages.append({"role": "user", "content": "i think it is order_12345"})
# // highlight-end

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    tools=tools
)

In [57]:
response

ChatCompletion(id='chatcmpl-A9mH6aTlDEAmmsM8T5zC9Sw0BXiqM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, I'd be happy to help with that. Could you please provide me with your order ID?", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893884, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=21, prompt_tokens=107, total_tokens=128, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [58]:
response.choices[0].message.content

"Sure, I'd be happy to help with that. Could you please provide me with your order ID?"

In [59]:
# Simulate the order_id and delivery_date
from datetime import datetime
order_id = "order_12345"
delivery_date = datetime.now()

# simulate the tool call response
response = {
    "choices": [
        {
            "message": {
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_62136354",
                        "type": "function",
                        "function": {
                            "arguments": "{'order_id': 'order_12345'}",
                            "name": "get_delivery_date"
                        }
                    }
                ]
            }
        }
    ]
}

function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "order_id": order_id,
        "delivery_date": delivery_date.strftime('%Y-%m-%d %H:%M:%S')
    }),
    "tool_call_id": response['choices'][0]['message']['tool_calls'][0]['id']
}

# chat completion payload
completion_payload = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
        {"role": "user", "content": "Hi, can you tell me the delivery date for my order?"},
        {"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"},
        {"role": "user", "content": "i think it is order_12345"},
        response['choices'][0]['message'],
        function_call_result_message
    ]
}

# call the openai api's chat completions endpoint
# to send the tool call result back to the model
response = openai.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)


In [60]:
response

ChatCompletion(id='chatcmpl-A9mHOGW3ogrbs66LLht9bPIbyEOlR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order (ID: order_12345) is scheduled to be delivered on **September 21, 2024, at 4:45 AM**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726893902, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3537616b13', usage=CompletionUsage(completion_tokens=49, prompt_tokens=131, total_tokens=180, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [61]:
response.choices[0].message.content

'Your order (ID: order_12345) is scheduled to be delivered on **September 21, 2024, at 4:45 AM**. If you have any other questions or need further assistance, feel free to ask!'